In [ ]:
!java -jar synthea-with-dependencies.jar -c synthea.properties -p 100 Minnesota

In [5]:
import autoTigerGraph as atg

In [4]:
!ls output/fhir

AllergyIntolerance.ndjson	 MedicationRequest.ndjson
CarePlan.ndjson			 Observation.ndjson
CareTeam.ndjson			 Organization.ndjson
Claim.ndjson			 Patient.ndjson
Condition.ndjson		 Practitioner.ndjson
Device.ndjson			 Procedure.ndjson
DiagnosticReport.ndjson		 SupplyDelivery.ndjson
Encounter.ndjson		 hospitalInformation1593855976157.json
ExplanationOfBenefit.ndjson	 hospitalInformation1594059421366.json
ImagingStudy.ndjson		 practitionerInformation1593855976157.json
Immunization.ndjson		 practitionerInformation1594059421366.json
MedicationAdministration.ndjson


In [7]:
atg.get_first('output/fhir/Patient.ndjson')

({'resourceType': 'Patient',
  'id': '8b179784-c6fb-4a7c-a671-930327c99090',
  'text': {'status': 'generated',
   'div': '<div xmlns="http://www.w3.org/1999/xhtml">Generated by <a href="https://github.com/synthetichealth/synthea">Synthea</a>.Version identifier: master-branch-latest-2-gb0629f3\n .   Person seed: -562926886057472933  Population seed: 1593855975766</div>'},
  'extension': [{'url': 'http://hl7.org/fhir/StructureDefinition/patient-mothersMaidenName',
    'valueString': 'Iva908 Tillman293'},
   {'url': 'http://hl7.org/fhir/StructureDefinition/patient-birthPlace',
    'valueAddress': {'city': 'Nagoya', 'state': ' Aichi', 'country': 'JP'}},
   {'url': 'http://synthetichealth.github.io/synthea/disability-adjusted-life-years',
    'valueDecimal': 0.005477761773323052},
   {'url': 'http://synthetichealth.github.io/synthea/quality-adjusted-life-years',
    'valueDecimal': 6.9945222382266765}],
  'identifier': [{'system': 'https://github.com/synthetichealth/synthea',
    'value': '

In [22]:
atg.get_first('output/fhir/Condition.ndjson')[0]

{'resourceType': 'Condition',
 'id': '8d31ba05-9fe0-4fcf-ba3b-81029b103881',
 'clinicalStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-clinical',
    'code': 'active'}]},
 'verificationStatus': {'coding': [{'system': 'http://terminology.hl7.org/CodeSystem/condition-ver-status',
    'code': 'confirmed'}]},
 'code': {'coding': [{'system': 'http://snomed.info/sct',
    'code': '74400008',
    'display': 'Appendicitis'}],
  'text': 'Appendicitis'},
 'subject': {'reference': 'Patient/6437da31-64bc-4849-93ef-91a3032d17ac'},
 'encounter': {'reference': 'Encounter/ddd5d359-43c5-407f-9616-450e1871e10f'},
 'onsetDateTime': '1988-01-07T18:14:53+00:00',
 'recordedDate': '1988-01-07T18:14:53+00:00'}

In [30]:
import ijson 
def get_field(filename, field_list, n):

    ids = []
    with open(filename, 'r') as f:
        objects = ijson.items(f, '', multiple_values=True, use_float=True)

        i = 0
        for json_object in objects:

            result=['']*len(field_list)
            
            if json_object:
                
                for j, field in enumerate(field_list):
                    result[j]=json_object
                    #print(result[j])
                    
                    for step in field:
                        result[j]= result[j][step]
                    
                ids.append(result)
                i += 1
                
                if i >= n:
                    return ids
                
    return ids

In [110]:
conditions = get_field('output/fhir/Condition.ndjson',
                       [['subject','reference'],
                        ['code','text'],
                        ['onsetDateTime']],
                       10**10)
conditions = [(condition[0][8:], condition[1], condition[2]) for condition in conditions if condition[1]=='COVID-19']
len(conditions)

178

In [111]:
conditions[:5]

[('6437da31-64bc-4849-93ef-91a3032d17ac',
  'COVID-19',
  '2020-03-05T19:23:53+00:00'),
 ('0cba9612-599a-422c-9f6d-8721e6bca920',
  'COVID-19',
  '2020-03-05T20:08:39+00:00'),
 ('46701b5c-8b5f-4b4e-92b0-d1027d5035ae',
  'COVID-19',
  '2020-03-13T11:11:12+00:00'),
 ('04574ef4-1df1-4bcf-bd1c-dccc65c243d9',
  'COVID-19',
  '2020-03-07T21:11:01+00:00'),
 ('8b179784-c6fb-4a7c-a671-930327c99090',
  'COVID-19',
  '2020-03-10T06:11:11+00:00')]

In [82]:
cond_list = set([condition[1] for condition in conditions])
patient_list = set([condition[0][8:] for condition in conditions])

In [100]:
import pandas as pd 
import numpy as np

df = pd.DataFrame(np.zeros((len(patient_list), len(cond_list))), 
                  index=patient_list, columns=cond_list)

date_df = pd.DataFrame([['']*len(cond_list)]*len(patient_list), 
                  index=patient_list, columns=cond_list)

In [101]:
df.head()

,Diabetes,Blighted ovum,Streptococcal sore throat (disorder),Smokes tobacco daily,Sprain of ankle,Small cell carcinoma of lung (disorder),Passive conjunctival congestion (finding),Sputum finding (finding),Tubal pregnancy,Escherichia coli urinary tract infection,...,Perennial allergic rhinitis,Acute bronchitis (disorder),Atopic dermatitis,History of appendectomy,Alcoholism,Sinusitis (disorder),"Localized, primary osteoarthritis of the hand",Third degree burn,Chronic kidney disease stage 1 (disorder),Appendicitis
-74f6-4c69-a35a-6f9c3a4e44f6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-59d7-45f0-b1c3-213fc51d0f69,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-63fc-4087-a86e-df59cecacd36,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-3a80-451c-ae22-ae2e09fc5e59,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
-30e7-4fba-b7c1-be60c8e065ee,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [102]:
date_df.head()

,Diabetes,Blighted ovum,Streptococcal sore throat (disorder),Smokes tobacco daily,Sprain of ankle,Small cell carcinoma of lung (disorder),Passive conjunctival congestion (finding),Sputum finding (finding),Tubal pregnancy,Escherichia coli urinary tract infection,...,Perennial allergic rhinitis,Acute bronchitis (disorder),Atopic dermatitis,History of appendectomy,Alcoholism,Sinusitis (disorder),"Localized, primary osteoarthritis of the hand",Third degree burn,Chronic kidney disease stage 1 (disorder),Appendicitis
-74f6-4c69-a35a-6f9c3a4e44f6,,,,,,,,,,,...,,,,,,,,,,
-59d7-45f0-b1c3-213fc51d0f69,,,,,,,,,,,...,,,,,,,,,,
-63fc-4087-a86e-df59cecacd36,,,,,,,,,,,...,,,,,,,,,,
-3a80-451c-ae22-ae2e09fc5e59,,,,,,,,,,,...,,,,,,,,,,
-30e7-4fba-b7c1-be60c8e065ee,,,,,,,,,,,...,,,,,,,,,,


In [103]:
for patient, condition, date in conditions:
    df.loc[patient,condition] = 1
    date_df.loc[patient,condition] = date

In [108]:
df[df['COVID-19']==1]

,Diabetes,Blighted ovum,Streptococcal sore throat (disorder),Smokes tobacco daily,Sprain of ankle,Small cell carcinoma of lung (disorder),Passive conjunctival congestion (finding),Sputum finding (finding),Tubal pregnancy,Escherichia coli urinary tract infection,...,Perennial allergic rhinitis,Acute bronchitis (disorder),Atopic dermatitis,History of appendectomy,Alcoholism,Sinusitis (disorder),"Localized, primary osteoarthritis of the hand",Third degree burn,Chronic kidney disease stage 1 (disorder),Appendicitis
6437da31-64bc-4849-93ef-91a3032d17ac,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,1.0
46701b5c-8b5f-4b4e-92b0-d1027d5035ae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
04574ef4-1df1-4bcf-bd1c-dccc65c243d9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
8b179784-c6fb-4a7c-a671-930327c99090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7a44b6a3-bee7-41d1-b570-8c596a2155f2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19856730-f11f-4e11-a2fe-a59a033f6ab0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN
cbc32084-01d1-4443-b23b-1162bbc900ef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d91b0d7e-8730-430b-ba99-147eaf491726,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bb7437cf-dc9a-4b45-ac3d-87c868bdf9bb,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
conditions[:5]

[('6437da31-64bc-4849-93ef-91a3032d17ac', 'Appendicitis'),
 ('1943a4dd-d6e4-42bb-a367-11d727d87127',
  'Body mass index 30+ - obesity (finding)'),
 ('47c464c6-9aa4-4544-bfb8-92cb021061b8', 'Coronary Heart Disease'),
 ('6437da31-64bc-4849-93ef-91a3032d17ac', 'History of appendectomy'),
 ('1943a4dd-d6e4-42bb-a367-11d727d87127', 'Chronic sinusitis (disorder)')]

In [79]:
pd.DataFrame(df.mean().sort_values(ascending=False), columns = ['Mean']).head(20)

,Mean
Suspected COVID-19,0.404396
COVID-19,0.396437
Fever (finding),0.375576
Viral sinusitis (disorder),0.374134
Cough (finding),0.335784
Acute viral pharyngitis (disorder),0.301546
Anemia (disorder),0.275401
Prediabetes,0.263587
Acute bronchitis (disorder),0.263587
Body mass index 30+ - obesity (finding),0.261580


In [80]:
df.shape

(542, 155)